In [1]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 구글 로그인 페이지로 이동
driver.get('https://accounts.google.com/signin')

# 수동으로 로그인 진행
print("Please log in manually and then press Enter here...")
input("Press Enter after you have logged in...")  # 터미널에서 엔터를 누름

# 로그인된 쿠키 저장
cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies saved.")
driver.quit()

Please log in manually and then press Enter here...
Cookies saved.


In [8]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_for_page_load(driver, timeout=50):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        return True
    except:
        return False

def scroll_to_element(driver, element):
    """ Scrolls the page until the element is visible """
    driver.execute_script("arguments[0].scrollIntoView(true); window.scrollBy(0, 1000);", element)  # Scroll a bit up to make sure the element is fully visible

def get_buttons(driver, button_selector, timeout=30):
    """ Retrieves buttons with a given selector and waits for them to be loaded """
    end_time = time.time() + timeout
    buttons = []
    while time.time() < end_time:
        buttons = driver.find_elements(By.CSS_SELECTOR, button_selector)
        if buttons:
            return buttons
        time.sleep(2)
    return buttons


def safe_click(driver, by, value, timeout=10):
    """ 안전하게 요소를 클릭하는 함수 """
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((by, value))
        )
        element.click()
    except Exception as e:
        print(f"요소 클릭 실패: {e}")

def safe_find_elements(driver, by, value, timeout=10):
    """ 안전하게 요소를 찾는 함수 """
    try:
        elements = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((by, value))
        )
        return elements
    except Exception as e:
        print(f"요소 찾기 실패: {e}")
        return []

# 기존 코드를 아래와 같이 변경
try:
    # 페이지 접속
    driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
    if not wait_for_page_load(driver):
        print("페이지 로딩에 실패했습니다.")
        driver.quit()
        exit()

    driver.maximize_window()
    time.sleep(3)

    # 데이터 저장 리스트 및 실패한 URL 저장 리스트
    all_data = {}
    failed_urls = []

    # URL 리스트 경로 (url 바꿔가면서 진행)
    path_list_file = r'D:\kdt_240424\workspace\project3\Crawling\URL_Crawling\Dog\설채현유튜브url모음\설채현의 수의학.csv'
    df = pd.read_csv(path_list_file)
   
    # url 몇개까지 할건지 개수지정. 앨런=하루 30개 제한, ex) [:30], [31:60], [61:90]...
    path_list = df['링크'].tolist()[0:20] 

    # 각 URL에 대해 반복 작업 수행
    for url in path_list:
        print(f"처리 중인 URL: {url}")
        driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
        if not wait_for_page_load(driver):
            print(f"URL {url}에 대한 페이지 로딩에 실패했습니다.")
            failed_urls.append(url)
            continue

        try:
            textarea = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'textarea[placeholder*="요약할 유튜브 링크를 입력해주세요"]'))
            )
            textarea.clear()
            textarea.send_keys(url)
            textarea.send_keys(Keys.RETURN)
            time.sleep(3)

            # 2. 로딩 대기
            WebDriverWait(driver, 200).until(
                EC.presence_of_element_located((By.XPATH, '//p[contains(text(), "영상 요약을 완료했습니다!")]'))
            )
            print(f"요약 완료 메시지가 표시되었습니다: {url}")
            time.sleep(10)

            # 3. 타이틀 추출
            titles = safe_find_elements(driver, By.CSS_SELECTOR, 'div._title_1skjv_47')
            print(f"타이틀 개수: {len(titles)}")
            time.sleep(5)

            # 4. 버튼 로딩 및 클릭
            buttons = safe_find_elements(driver, By.CSS_SELECTOR, 'button._btn_arrow_1skjv_57')
            if not buttons:
                print("버튼이 로딩되지 않았습니다.")
                continue
            print(f"버튼 개수: {len(buttons)}")
            time.sleep(5)

            url_data = {
                'URL': url,
                'SUMMARIES': []
            }

            for i in range(len(buttons)):
                try:
                    # 버튼을 다시 찾아서 클릭
                    buttons = safe_find_elements(driver, By.CSS_SELECTOR, 'button._btn_arrow_1skjv_57')
                    scroll_to_element(driver, buttons[i])
                    ActionChains(driver).move_to_element(buttons[i]).click().perform()
                    print(f"버튼 클릭됨: {i}")
                    time.sleep(3)

                    # '자세히' 버튼 로딩 및 클릭
                    detail_buttons = safe_find_elements(driver, By.CSS_SELECTOR, 'button._btn_detail_rtro1_37')
                    if i < len(detail_buttons):
                        scroll_to_element(driver, detail_buttons[i])
                        ActionChains(driver).move_to_element(detail_buttons[i]).click().perform()
                        print(f"'자세히' 버튼 클릭됨: {i}")
                        time.sleep(5)  # 버튼 클릭 후 로딩 시간 대기

                        # 타이틀 추출
                        titles = safe_find_elements(driver, By.CSS_SELECTOR, 'div._title_1skjv_47')
                        summary_title = titles[i].text if i < len(titles) else f"Summary {i+1}"
                        print(f"타이틀 추출됨: {summary_title}")

                        # 간단 텍스트 모음 추출
                        simple_text_elements = safe_find_elements(driver, By.XPATH, '//ul[@class="_ul_tnadh_1"]/li/span/p[@class="_p_qdrcw_1"]')
                        simple_text = "\n".join([element.text for element in simple_text_elements])
                        print(f"간단 텍스트 추출됨: {simple_text}")

                        # 자세히 텍스트 모음 추출
                        detail_text_elements = safe_find_elements(driver, By.XPATH, '//div[@class="_detail_rtro1_31"]/p[@class="_p_qdrcw_1"]')
                        detail_text = "\n".join([element.text for element in detail_text_elements])
                        print(f"자세히 텍스트 추출됨: {detail_text}")

                        # 추출한 데이터를 저장
                        url_data['SUMMARIES'].append({
                            'SUMMARY_TITLE': summary_title,
                            'SIMPLE_TEXT': simple_text,
                            'DETAIL_TEXT': detail_text
                        })

                except Exception as e:
                    print(f"요약본 처리 오류: {e}")
                    continue

            all_data[url] = url_data

        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            failed_urls.append(url)
            continue

    # JSON 파일로 저장
    try:
        with open('extracted_data.json', 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)
        print("데이터가 extracted_data.json 파일에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}")

    # 실패한 URL 저장 (선택 사항)
    if failed_urls:
        with open('failed_urls.txt', 'w') as f:
            for url in failed_urls:
                f.write(url + '\n')

finally:
    # 드라이버 종료
    driver.quit()


처리 중인 URL: https://www.youtube.com/watch?v=sh74Z0EPdHU&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=1&pp=iAQB
요약 완료 메시지가 표시되었습니다: https://www.youtube.com/watch?v=sh74Z0EPdHU&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=1&pp=iAQB
타이틀 개수: 2
버튼 개수: 2
버튼 클릭됨: 0
'자세히' 버튼 클릭됨: 0
타이틀 추출됨: 챌린지 도전과 성공
간단 텍스트 추출됨: 챌린지 도전에 대한 소개와 성공적인 결과를 공유하며, 반려견의 행복을 위한 앱 출시 소식을 전합니다.
유튜브 이름 변경과 회사를 설립한 이유는 반려견의 행복을 증진시키기 위함입니다.
앱은 반려견과 보호자가 함께 즐길 수 있는 다양한 놀이를 제공하며, 집에서도 활동적인 시간을 보낼 수 있도록 돕습니다.
자세히 텍스트 추출됨: 챌린지에서 도전 결과를 확인해 주세요.
첫 도전 성공을 해서 500 트릿을 더 받아서 3,800 트릿이 됐습니다.
안녕하세요, 설치 여러분. 오늘 제가 소개해 드릴 것은 제가 그동안 엄청난 노력을 해서 꼭 만들고 싶었던 것입니다.
제 유튜브 이름이 놀러와로 바뀌었습니다.
우리 반려견들이 조금이라도 더 행복했으면 좋겠다는 생각으로 이 회사를 만들었습니다.
여러분들과 우리 반려견들에게 즐거움과 행복을 줄 결정체 물로 플레이 앱이 드디어 출시됐습니다.
제가 왜 이 앱을 만들고 싶었는지 말씀드리겠습니다.
우리 반려견들이 집에서 편히 쉬는 경우도 있지만, 많은 반려견들이 집에서 지루하게 지내는 경우가 많습니다.
그렇기 때문에 집 안에서도 즐거운 놀이가 필요하다고 생각했습니다.
이 앱은 반려견과 보호자 모두가 함께 즐길 수 있도록 만들어졌습니다.
버튼 클릭됨: 1
'자세히' 버튼 클릭됨: 1
타이틀 추출됨: 앱 기능 소개
간단 텍스트 추출됨: 챌린지 도전에 대한 소개와 성공적인 결과를 공유하며, 반려견의 행복을

In [7]:
import pandas as pd
path_list_file = r'D:\kdt_240424\workspace\project3\Crawling\URL_Crawling\Dog\설채현유튜브url모음\설채현의 수의학.csv'
df = pd.read_csv(path_list_file)

path_list = df['링크'].tolist()[0:20]
print(path_list)
# path_list의 길이(즉, URL의 개수)를 출력
print(f'Number of URLs in path_list: {len(path_list)}')

# path_list의 마지막 요소를 출력
print(f'Last URL in path_list: {path_list[-1]}')

['https://www.youtube.com/watch?v=sh74Z0EPdHU&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=1&pp=iAQB', 'https://www.youtube.com/watch?v=ZHpM_Scrpxk&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=2&pp=iAQB', 'https://www.youtube.com/watch?v=pdeMGKLAUkM&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=3&pp=iAQB', 'https://www.youtube.com/watch?v=Mq_WN5BRWWk&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=4&pp=iAQB', 'https://www.youtube.com/watch?v=UyjcuBZIv68&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=5&pp=iAQB', 'https://www.youtube.com/watch?v=93xZNcmwdOg&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=6&pp=iAQB', 'https://www.youtube.com/watch?v=92EByzD_Dt4&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=7&pp=iAQB', 'https://www.youtube.com/watch?v=7t6sA9edH2Q&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=8&pp=iAQB', 'https://www.youtube.com/watch?v=ADoEADVaK3k&list=PLfLVnVofgzHKAkaEjwqsLtBFeohxkcsK9&index=9&pp=iAQB', 'https://www.youtube.com/watch?v=4AaT3UeEWgA&list=PLfLVnVofgzHKAkaEjwqsL